In [1]:
import sys
sys.path.append("../../src")
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn.functional as F

import glob
import os
from datetime import datetime
import time
import math
from tqdm import tqdm

from itertools import repeat
from torch.nn.parameter import Parameter
import collections
import matplotlib
from torch_utils import *
from utils import display_matrix
from debug import *
from visualization import *
# matplotlib.use('Agg')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), 
                                            torchvision.transforms.Normalize(mean=(0.0,), std=(1.0,))])

mnist_dset_train = torchvision.datasets.MNIST('../../data', train=True, transform=transform, target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(mnist_dset_train, batch_size=20, shuffle=True, num_workers=0)

mnist_dset_test = torchvision.datasets.MNIST('../../data', train=False, transform=transform, target_transform=None, download=True)
test_loader = torch.utils.data.DataLoader(mnist_dset_test, batch_size=20, shuffle=False, num_workers=0)

In [4]:
activation = hard_sigmoid
architecture = [784, 500, 10]

x,y = next(iter(train_loader))
x = x.view(x.size(0),-1).to(device).T
y_one_hot = F.one_hot(y, 10).to(device).T

beta = 2
lambda_ = 0.99995
epsilon = 0.15
one_over_epsilon = 1 / epsilon
lr_start = {'ff' : np.array([0.015, 0.015]), 'fb': np.array([0.01, 0.001])}

neural_lr_start = 0.1
neural_lr_stop = 0.001
neural_lr_rule = "constant"
neural_lr_decay_multiplier = 0.01
neural_dynamic_iterations_nudged = 10
neural_dynamic_iterations_free = 20
hopfield_g = 0.05

model = CorInfoMaxHopfield(architecture = architecture, lambda_ = lambda_, 
                                      epsilon = epsilon, activation = activation)

In [5]:
# neurons[0].shape

In [6]:
# def append_ones_row_vector_to_tensor(inp_vector):
#     return torch.cat((inp_vector, torch.ones(1, inp_vector.shape[1]).to(inp_vector.device)), 0)

In [7]:
# x, y = next(iter(train_loader))
# x, y = x.to(device), y.to(device)
# x = x.view(x.size(0),-1).T
# y_one_hot = F.one_hot(y, 10).to(device).T
# print(model.B[0]['weight'][:-1].shape)
# neurons = model.init_neurons(x.size(1), device = model.device)

# torch.cat((neurons[0], torch.ones(1, neurons[0].shape[1]).to(device)), 0).shape
# append_ones_row_vector_to_tensor(neurons[0]).shape

In [8]:
_ = evaluatePC(model, train_loader, device, False)

Train accuracy :	 0.14398333333333332


In [9]:
trn_acc_list = []
tst_acc_list = []

n_epochs = 20

for epoch_ in range(n_epochs):
    lr = {'ff' : lr_start['ff'] * (0.99)**epoch_, 'fb' : lr_start['fb'] * (0.99)**epoch_}
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device), y.to(device)
        x = x.view(x.size(0),-1).T
        y_one_hot = F.one_hot(y, 10).to(device).T
        
        take_debug_logs_ = (idx % 500 == 0)
        if True:
            rnd_sgn = 2*np.random.randint(2) - 1
            beta = rnd_sgn*beta
            
        neurons = model.batch_step_hopfield( x, y_one_hot, hopfield_g, 
                                             lr, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                                             neural_lr_decay_multiplier, neural_dynamic_iterations_free,
                                             neural_dynamic_iterations_nudged, beta, take_debug_logs_)
    
    trn_acc = evaluatePC(model, train_loader, device, False, printing = False)
    tst_acc = evaluatePC(model, test_loader, device, False, printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))

3000it [00:33, 89.29it/s]
9it [00:00, 84.18it/s]

Epoch : 1, Train Accuracy : 0.79995, Test Accuracy : 0.8116


3000it [00:34, 86.12it/s]
9it [00:00, 84.98it/s]

Epoch : 2, Train Accuracy : 0.8361833333333333, Test Accuracy : 0.8414


3000it [00:34, 86.13it/s]
9it [00:00, 85.32it/s]

...

Epoch : 16, Train Accuracy : 0.8476333333333333, Test Accuracy : 0.8519


3000it [00:34, 87.80it/s]
9it [00:00, 84.81it/s]

Epoch : 17, Train Accuracy : 0.8479333333333333, Test Accuracy : 0.8528


744it [00:08, 87.06it/s]


KeyboardInterrupt: 

In [10]:
trn_acc = evaluateContrastiveCorInfoMaxHopfield(model, train_loader, hopfield_g, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                                                neural_lr_decay_multiplier, neural_dynamic_iterations_free, device, printing = False)
tst_acc = evaluateContrastiveCorInfoMaxHopfield(model, test_loader, hopfield_g, neural_lr_start, neural_lr_stop, neural_lr_rule, 
                                                neural_lr_decay_multiplier, neural_dynamic_iterations_free, device, printing = False)


In [11]:
trn_acc, tst_acc

(0.09871666666666666, 0.098)

In [ ]:
plot_convergence_plot(trn_acc_list, xlabel = 'Number of Epochs', ylabel = 'Accuracy %',
                      title = 'Contrastive CorInfoMax Train Accuracy w.r.t. Epochs', 
                      figsize = (12,8), fontsize = 25, linewidth = 3)

In [ ]:
plot_convergence_plot(tst_acc_list, xlabel = 'Number of Epochs', ylabel = 'Accuracy %',
                      title = 'Contrastive CorInfoMax Test Accuracy w.r.t. Epochs', 
                      figsize = (12,8), fontsize = 25, linewidth = 3)

In [ ]:
plt.figure(figsize = (18, 12))
plt.plot(model.forward_backward_angles, linewidth=2)
plt.xlabel("Number of Debug Points", fontsize=45)
plt.ylabel("Angle (Degree)", fontsize=45)
plt.title("Angles of Forward and Backward Weights", fontsize=45)
plt.grid()
plt.xticks(fontsize=45)
plt.yticks(fontsize=45)
plt.show()

In [ ]:
display_matrix(model.B[0]['weight'][:10,:10])

In [ ]:
display_matrix(torch.linalg.inv(model.R[0]['weight'])[:10,:10])

In [ ]:
# from torch_utils import *

In [ ]:
# def angle_between_two_matrices(A, B):
#     """Computes the angle between two matrices A and B.

#     Args:
#         A (torch.Tensor): Pytorch tensor of size m times n
#         B (torch.Tensor): Pytorch tensor of size m times n

#     Returns:
#         angle: angle between the matrices A and B. The formula is given by the following:
#                (180/pi) * acos[ Tr(A @ B.T) / sqrt(Tr(A @ A.T) * Tr(B @ B.T))] 
#     """

#     angle = (180 / torch.pi) * torch.acos(torch.trace(A @ B.T) / torch.sqrt(torch.trace(A @ A.T) * torch.trace(B @ B.T)))
#     return angle

In [ ]:
plt.plot(model.forward_backward_angles)

In [ ]:
angle_between_two_matrices(model.Wff[1]['weight'], model.Wfb[1]['weight'].T).item()

In [ ]:
# torch.norm(model.B[0]['weight'] - torch.linalg.inv(model.Rh1))

In [ ]:
# display_matrix(torch.linalg.inv(model.Rh2)[:10,:10])